#### Imports

In [20]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


#### SKIP the following cells and start from the line where we read "ny_nj_connectedness_final" , if you already have the "ny_nj_connectedness_final" graph

In [48]:
ny_nj_graph = nx.read_gexf("ny_nj_connectedness.gexf")



In [50]:
ny_nj_graph.nodes['7002']

{'label': '7002'}

In [ ]:
ny_nj_zipcodes = list(range(10001, 14976)) + list(range(7001, 8980))


#### include mean income from 2018
 


In [ ]:
incomes = pd.read_csv("zip_inc_final.csv",delimiter=',')
#Removing non numeric values and "_" values from the mean_income column
incomes['Mean_Income'] = incomes['Mean_Income'].replace(['-','N'], np.nan)
incomes['Mean_Income'] = pd.to_numeric(incomes['Mean_Income'])
print(incomes['Mean_Income'].dtype)
incomes = incomes.dropna(subset=['Mean_Income'])


# Filtering just the ny-nj state zipcodes
incomes_ny_nj = incomes[incomes['zip'].isin(ny_nj_zipcodes)]

zip_income_dict_ny_nj ={}
for _, row in incomes_ny_nj.iterrows():
    zip_income_dict_ny_nj[row['zip']] = row['Mean_Income']

#### Assign mean income , and the ses values based on mean income based on the zipwise mean income dict

In [ ]:
incomes_ny_nj_median_income = incomes_ny_nj['Mean_Income'].median()
print(incomes_ny_nj_median_income)

nodes_to_remove = []
# Add the mean income and zip as node attributes, and assign the economic
# connectedness - ses ('low' if below median income.. else 'high')
for node in ny_nj_graph.nodes():
    if int(node) in zip_income_dict_ny_nj:
        ny_nj_graph.nodes[node]['mean_income'] = zip_income_dict_ny_nj[int(node)]
        if ny_nj_graph.nodes[node]['mean_income']>=incomes_ny_nj_median_income:
            ny_nj_graph.nodes[node]['ses'] = "high"
        else:
            ny_nj_graph.nodes[node]['ses'] = "low"
    else:
        nodes_to_remove.append(node)
    ny_nj_graph.nodes[node]['zip'] = node
    
#Removing nodes without a mean income data
for node in nodes_to_remove:
    ny_nj_graph.remove_node(node)
    
print(ny_nj_graph.nodes['7002']['mean_income'] , ny_nj_graph.nodes['7002']['ses'])


### Calculate economic connectedness based on ses:
Economic connectedness (EC) is 
calculated as the share of friends with above-median SES (‘high SES’) among 
people with below-median SES (‘low SES’) divided by 50%. This measures 
the average degree of under-representation of high-SES friends among people with low SES.

In [ ]:
# EC = (number of high-SES friends / total friends) / 0.5

for node in ny_nj_graph.nodes():
    hi_ses_weight = 0
    total_weight = 0 #Includes lo and hi ses weights
    ec = 0
    neighbours = list(ny_nj_graph.neighbors(node))
    for neighbour in neighbours:
        edge_weight = ny_nj_graph.get_edge_data(node, neighbour)['weight']
        neighbour_ses = ny_nj_graph.nodes[neighbour]["ses"]
        if neighbour_ses == 'low':
            total_weight+=edge_weight
        else:
            hi_ses_weight+=edge_weight
    total_weight+=hi_ses_weight
    ec = (hi_ses_weight/total_weight)/0.5
    ny_nj_graph.nodes[node]['ec'] = ec
    
        
            
            

### Calculate the mobility percentile and assign to the graph 

In [ ]:
#Get the zipcode wise mobility data for the states in consideration
mobility_data = pd.read_stata('/content/drive/MyDrive/Colab Notebooks/ns_project/zip_mobility_data.dta')
mobility_data_ny_nj = mobility_data[mobility_data['zip'].isin(ny_nj_zipcodes)]
mobility_data_ny_nj = mobility_data_ny_nj.dropna(subset=['kfr_pooled_pooled_p25'])

#Since the mean income percentile might not tell the complete story. We are calculating the percentile of performance in terms of the state we have selected
mobility_data_ny_nj['Mobility_Percentile'] = mobility_data_ny_nj.kfr_pooled_pooled_p25.rank(pct = True)

#creating a dictionary of zip as key and mobility percentile as value
mobility_dict_ny_nj ={}
for _, row in mobility_data_ny_nj.iterrows():
    mobility_dict_ny_nj[row['zip']] = row['Mobility_Percentile']
    
#pruning the low ses nodes with no mobility data
nodes_to_remove = []
# Removing low SES nodes, which doesn't have a mobility data
for node in ny_nj_graph.nodes():
    if int(node) in mobility_dict_ny_nj:
        ny_nj_graph.nodes[node]['Mobility_Percentile'] = mobility_dict_ny_nj[int(node)]
    else:
        nodes_to_remove.append(node)
        # ny_nj_graph.nodes[node]['Mobility_Percentile'] = 0
        
for node in nodes_to_remove:
    ny_nj_graph.remove_node(node)

nx.write_gexf(ny_nj_graph, 'ny_nj_connectedness_final.gexf')









### ------->> Note: Start here if you already have the "ny_nj_connectedness_final" graph

### Read the final graph with mobility percentile, ses and mean incomes as node attributes


In [51]:
ny_nj_graph = nx.read_gexf("ny_nj_connectedness_final.gexf")
edges = len(ny_nj_graph.edges())
nodes = len(ny_nj_graph.nodes())


print("edges:: ", edges,"\n nodes",nodes)

edges::  1427444 
 nodes 1923


In [52]:
ny_nj_graph.nodes['7002']

{'mean_income': 35905.0,
 'ses': 'low',
 'zip': '7002',
 'ec': 1.3331180464931969,
 'Mobility_Percentile': 0.6700636942675159,
 'label': '7002'}

In [23]:
ny_nj_graph.edges['7002','7003']

{'id': '1', 'weight': 274213.0}

### The below cell is only for visualization. we are removing some edges based on the threshold

In [ ]:

# set the threshold for edge weight
threshold = 100000

# create a new graph with only the edges that have weight above the threshold
new_edges = [(u, v, d) for u, v, d in ny_nj_graph.edges(data=True) if d['weight'] >= threshold]
new_graph = nx.Graph(new_edges)

# copy over the node attributes from the original graph to the new graph
for node in ny_nj_graph.nodes:
    new_graph.nodes[node].update(ny_nj_graph.nodes[node])

# the new graph has only the important edges and the same node attributes as the original graph

In [ ]:
edges = len(new_graph.edges())
nodes = len(new_graph.nodes())


print("edges:: ", edges,"\n nodes",nodes)

In [ ]:
nx.write_gexf(new_graph,"ny_nj_graph_edges_pruned.gexf")

In [ ]:
ny_nj_graph.nodes['7002']

### code for geo plot

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
#ny_nj_graph = nx.read_gexf("/content/drive/MyDrive/Colab Notebooks/ns_project/ny_nj_connectedness_final.gexf")
ny_nj_ziplist = list(ny_nj_graph.nodes)
ny_nj_zipcodes = gpd.GeoDataFrame()
print((ny_nj_ziplist))
#all_zipcodes = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/ns_project/zip_shapes/tl_2022_us_zcta520.shx')

for zip in ny_nj_ziplist:
  #print(zip)
  if zip in (all_zipcodes['ZCTA5CE20'].values):
    ny_nj_zipcodes = pd.concat([ny_nj_zipcodes, all_zipcodes[all_zipcodes['ZCTA5CE20'] == zip]])
ny_nj_zipcodes.plot(column='ZCTA5CE20', cmap='viridis')
plt.show()

### Below cells are used to remove the low ses - low ses edges

In [54]:
ny_nj_graph_edges_removed_lo_ses = ny_nj_graph.copy()

In [55]:
edges_to_remove = []
# Assuming you have a networkx graph object named 'G'
for u, v, data in ny_nj_graph_edges_removed_lo_ses.edges(data=True):
    # Check if both nodes have 'ses' attribute and their values are "low"
    if ny_nj_graph_edges_removed_lo_ses.nodes[u].get('ses') == 'low' and ny_nj_graph_edges_removed_lo_ses.nodes[v].get('ses') == 'low':
        # Remove the edge between u and v
        edges_to_remove.append((u,v))
for u,v in edges_to_remove:
        ny_nj_graph_edges_removed_lo_ses.remove_edge(u,v)

In [56]:
#Comapring the number of edges between the original graph and the graph with lo_ses - low_ses edges removed 

print(len(ny_nj_graph.edges()))
print(len(ny_nj_graph_edges_removed_lo_ses.edges()))

1427444
1136202


### The cells below are used to calculate all the centrality measures

In [57]:
#PageRank
pr = nx.pagerank(ny_nj_graph_edges_removed_lo_ses,weight = "weight")

In [58]:
# Eigen vector
ev = nx.eigenvector_centrality(ny_nj_graph_edges_removed_lo_ses, weight="weight")

In [64]:
#Betweenness centrality - taking too long to compute
# bc = nx.betweenness_centrality(ny_nj_graph_edges_removed_lo_ses,weight="weight")

### Add the lat and long as attributes for the xipcodes

In [70]:
ny_nj_zipcodes = list(range(10001, 14976)) + list(range(7001, 8980))

lat_long = pd.read_csv('zip_lat_long.txt')
nynj_lat_long = lat_long[lat_long['ZIP'].isin(ny_nj_zipcodes)]
zip_lat_dict = {}
zip_long_dict = {}
for _, row in nynj_lat_long.iterrows():
    zip_lat_dict[row['ZIP']] = row['LAT']
    zip_long_dict[row['ZIP']] = row['LNG']
for node in ny_nj_graph_edges_removed_lo_ses.nodes():
    if int(node) in zip_lat_dict:
        ny_nj_graph_edges_removed_lo_ses.nodes[node]['latitude'] = zip_lat_dict[int(node)]
        ny_nj_graph_edges_removed_lo_ses.nodes[node]['longitude'] = zip_long_dict[int(node)]
        
        #The original graph
        ny_nj_graph.nodes[node]['latitude'] = zip_lat_dict[int(node)]
        ny_nj_graph.nodes[node]['longitude'] = zip_long_dict[int(node)]
print(nx.info(ny_nj_graph_edges_removed_lo_ses))

Graph with 1923 nodes and 1136202 edges


/var/folders/sr/swqxjh5n7sb5rklv1yzdl41m0000gn/T/ipykernel_48532/445799950.py:18: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(ny_nj_graph_edges_removed_lo_ses))


### Add all the calculated centralities as node attributes

In [69]:
nx.set_node_attributes(ny_nj_graph_edges_removed_lo_ses,pr,"Pagerank")
nx.set_node_attributes(ny_nj_graph_edges_removed_lo_ses,ev,"Eigenvector")

##The original graph
nx.set_node_attributes(ny_nj_graph,pr,"Pagerank")
nx.set_node_attributes(ny_nj_graph,ev,"Eigenvector")

In [62]:
ny_nj_graph_edges_removed_lo_ses.nodes['7002']

{'mean_income': 35905.0,
 'ses': 'low',
 'zip': '7002',
 'ec': 1.3331180464931969,
 'Mobility_Percentile': 0.6700636942675159,
 'label': '7002',
 'latitude': 40.662338,
 'longitude': -74.110275,
 'Pagerank': 0.00024845809325711667,
 'Eigenvector': 0.00041792706419122445}

In [66]:
nx.write_gexf(ny_nj_graph_edges_removed_lo_ses,"ny_nj_connectedness_final_allattributes.gexf")

In [74]:
nx.write_gexf(ny_nj_graph,"ny_nj_connectedness_final_allattributes_original.gexf")

#### Just for visualization on gephi, pruning some edges based on threshold
For visualization, lets use the original ny_nj_graph , instead of the lo-ses-lo_ses removed edges one, since we already have the metrics calculated.

In [73]:
# set the threshold for edge weight
threshold = 100000

# create a new graph with only the edges that have weight above the threshold
new_edges = [(u, v, d) for u, v, d in ny_nj_graph.edges(data=True) if d['weight'] >= threshold]
new_graph = nx.Graph(new_edges)

# copy over the node attributes from the original graph to the new graph
for node in ny_nj_graph.nodes:
    new_graph.nodes[node].update(ny_nj_graph.nodes[node])

# the new graph has only the important edges and the same node attributes as the original graph

edges = len(new_graph.edges())
nodes = len(new_graph.nodes())


print("edges:: ", edges,"\n nodes",nodes)
nx.write_gexf(new_graph,"ny_nj_connectedness_final_allattributes_pruned.gexf")

edges::  292525 
 nodes 1923


### Calculate clustering coefficient and assortativity

In [76]:
# #Taking too long to compute
# clustering_coeff_nynj = nx.clustering(ny_nj_graph_edges_removed_lo_ses,weight="weight")
# assortativity_njnj = nx.assortativity(ny_nj_graph_edges_removed_lo_ses,weight="weight")

### Show top nodes in terms of page rank

In [85]:

# Sort the nodes by PageRank score in descending order
pr_sorted = sorted(ev.items(), key=lambda x: x[1], reverse=True)

# Display the top 10 nodes with their corresponding ses values
print("Top 10 nodes by PageRank:")
for i in range(10):
    node = pr_sorted[i][0]
    pr_score = pr_sorted[i][1]
    ses_value = ny_nj_graph.nodes[node]['ses']
    print(f"Node {node}: PageRank score = {pr_score:.4f}, ses value = {ses_value}")


Top 10 nodes by PageRank:
Node 12413: PageRank score = 0.1872, ses value = high
Node 12473: PageRank score = 0.1850, ses value = high
Node 12431: PageRank score = 0.1724, ses value = high
Node 12037: PageRank score = 0.1657, ses value = high
Node 12046: PageRank score = 0.1542, ses value = high
Node 12521: PageRank score = 0.1484, ses value = high
Node 12015: PageRank score = 0.1462, ses value = high
Node 12184: PageRank score = 0.1420, ses value = high
Node 12143: PageRank score = 0.1396, ses value = high
Node 12463: PageRank score = 0.1374, ses value = high


In [ ]:
cal_graph = nx.read_gexf("/content/drive/MyDrive/Colab Notebooks/ns_project/ny_nj_connectedness_final.gexf")
cal_ziplist = list(cal_graph.nodes)
cal_zipcodes = gpd.GeoDataFrame()
print((cal_ziplist))
for zip in cal_ziplist:
  #print(zip)
  if zip in (all_zipcodes['ZCTA5CE20'].values):
    cal_zipcodes = pd.concat([cal_zipcodes, all_zipcodes[all_zipcodes['ZCTA5CE20'] == zip]])
	
cal_df = pd.DataFrame(columns=['zip','ses','ec','Mobility_Percentile'])
ec = nx.get_node_attributes(cal_df, 'ec')
ec_array = np.array(list(ec.values()))
median_ec2 = np.median(ec_array)
#implementation
for node in cal_graph.nodes():
    data = [node,cal_graph.nodes[node]['ses'],cal_graph.nodes[node]['ec'],cal_graph.nodes[node]['Mobility_Percentile']]
    data_df = pd.DataFrame([data], columns=['zip', 'ses','ec','Mobility_Percentile'])
    cal_df = pd.concat([cal_df,data_df],ignore_index=True)
print(cal_df)
for i in range(len(cal_df)):
    if cal_df.iloc[i]['ec'] >= median_ec2 and cal_df.iloc[i]['Mobility_Percentile'] >= 0.5:
        cal_df.loc[i, 'color'] = 'Green'
    elif cal_df.iloc[i]['ec'] < median_ec2 and cal_df.iloc[i]['Mobility_Percentile'] < 0.5:
        cal_df.loc[i, 'color'] = 'Green'
    else:
        cal_df.loc[i, 'color'] = 'Red'
cal_zipcodes = cal_zipcodes.rename(columns={'ZCTA5CE20': 'zip'})
merged2 = cal_zipcodes.merge(ny_nj_df, on='zip')
merged2.plot(color = merged2['color'])
plt.show()